In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Kidney-Disease-Classification\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\Kidney-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration for data ingestion.

    Attributes:
        root_dir (Path): The root directory for data ingestion.
        source_dir (str): The source directory or URL for data retrieval.
        local_data_file (Path): The local file path for downloaded data.
        unzip_dir (Path): The directory where data will be extracted or stored after ingestion.
    """
    root_dir: Path
    source_dir: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import  *

In [7]:
from pathlib import Path

class ConfigurationManager:
    def __init__(self, 
                config_filepath: Path = None, 
                params_filepath: Path = None):
        """
        Initialize the ConfigurationManager.

        Args:
            config_filepath (Path, optional): The path to the configuration file.
            params_filepath (Path, optional): The path to the parameters file.
        """
        if config_filepath is None:
            config_filepath = CONFIG_FILE_PATH

        if params_filepath is None:
            params_filepath = PARAMS_FILE_PATH

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create directories if needed
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Get the data ingestion configuration.

        Returns:
            DataIngestionConfig: The data ingestion configuration.
        """
        config = self.config.data_ingestion

        # Create directories if needed
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_dir=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import zipfile
import gdown
from cnnClassifier import configure_logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize the DataIngestion instance.

        Args:
            config (DataIngestionConfig): Configuration for data ingestion.
        """
        self.config = config
        self.logger = configure_logger("logs", "data_ingestion.log")

    def download_file(self) -> None:
        """
        Download data from a URL and save it to the local file.

        Raises:
            Exception: If an error occurs during the download.
        """
        try:
            dataset_url = self.config.source_dir
            zip_download_dir = self.config.local_data_file

            os.makedirs('artifacts/data_ingestion', exist_ok=True)

            self.logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            self.logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            self.logger.error(f"Error during data download: {str(e)}")
            raise e

    def extract_zip_file(self) -> None:
        """
        Extract the contents of a ZIP file to a specified directory.

        Raises:
            Exception: If an error occurs during the extraction.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-15 17:55:28] [cnnClassifierLogger] [INFO]: [common.py]: YAML file 'config\config.yaml' loaded successfully.
[2023-10-15 17:55:28] [cnnClassifierLogger] [INFO]: [common.py]: YAML file 'params.yaml' loaded successfully.
[2023-10-15 17:55:28] [cnnClassifierLogger] [INFO]: [common.py]: Created directory at: artifacts
[2023-10-15 17:55:28] [cnnClassifierLogger] [INFO]: [common.py]: Created directory at: artifacts/data_ingestion
[2023-10-15 17:55:28] [cnnClassifierLogger] [INFO]: [2869915300.py]: Downloading data from https://drive.google.com/file/d/1Cbowka-nMgOAQIL_N3NVWBpo4eioCf8O/view?usp=sharing into file artifacts/data_ingestion/data.7z
[2023-10-15 17:55:28] [cnnClassifierLogger] [INFO]: [2869915300.py]: Downloading data from https://drive.google.com/file/d/1Cbowka-nMgOAQIL_N3NVWBpo4eioCf8O/view?usp=sharing into file artifacts/data_ingestion/data.7z


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1Cbowka-nMgOAQIL_N3NVWBpo4eioCf8O
From (redirected): https://drive.google.com/uc?/export=download&id=1Cbowka-nMgOAQIL_N3NVWBpo4eioCf8O&confirm=t&uuid=c6481a3f-7cbc-4fa9-97e4-c7ada5d34cec
To: c:\Users\DELL\OneDrive\Desktop\Kidney-Disease-Classification\artifacts\data_ingestion\data.7z
100%|██████████| 940M/940M [06:40<00:00, 2.35MB/s] 

[2023-10-15 18:02:12] [cnnClassifierLogger] [INFO]: [2869915300.py]: Downloaded data from https://drive.google.com/file/d/1Cbowka-nMgOAQIL_N3NVWBpo4eioCf8O/view?usp=sharing into file artifacts/data_ingestion/data.7z
[2023-10-15 18:02:12] [cnnClassifierLogger] [INFO]: [2869915300.py]: Downloaded data from https://drive.google.com/file/d/1Cbowka-nMgOAQIL_N3NVWBpo4eioCf8O/view?usp=sharing into file artifacts/data_ingestion/data.7z
